<a href="https://colab.research.google.com/github/rameshbabulakshmanan84/AI-Day---Knowledge-Graph/blob/master/BERT-NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import pandas as pd
path='/content/drive/My Drive/NLP/ner - custom.csv'
data=pd.read_csv(path)
data.tail(10)

,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,pos,prev-iob,prev-lemma,prev-pos,prev-prev-iob,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
1012,1012,",",a,black,JJ,lowercase,black,DT,lowercase,a,",",O,fatigu,NNS,O,in,IN,lowercase,in,lowercase,fatigues,43,punct,",",O
1013,1013,a,black,cap,NN,lowercase,cap,JJ,lowercase,black,DT,O,",",",",O,fatigu,NNS,lowercase,fatigues,punct,",",43,lowercase,a,O
1014,1014,black,cap,",",",",punct,",",NN,lowercase,cap,JJ,O,a,DT,O,",",",",punct,",",lowercase,a,43,lowercase,black,O
1015,1015,cap,",",and,CC,lowercase,and,",",punct,",",NN,O,black,JJ,O,a,DT,lowercase,a,lowercase,black,43,lowercase,cap,O
1016,1016,",",and,a,DT,lowercase,a,CC,lowercase,and,",",O,cap,NN,O,black,JJ,lowercase,black,lowercase,cap,43,punct,",",O
1017,1017,and,a,bulki,JJ,lowercase,bulky,DT,lowercase,a,CC,O,",",",",O,cap,NN,lowercase,cap,punct,",",43,lowercase,and,O
1018,1018,a,bulki,belt,NN,lowercase,belt,JJ,lowercase,bulky,DT,O,and,CC,O,",",",",punct,",",lowercase,and,43,lowercase,a,O
1019,1019,bulki,belt,.,.,punct,.,NN,lowercase,belt,JJ,O,a,DT,O,and,CC,lowercase,and,lowercase,a,43,lowercase,bulky,O
1020,1020,belt,.,__end1__,__END1__,wildcard,__END1__,.,punct,.,NN,O,bulki,JJ,O,a,DT,lowercase,a,lowercase,bulky,43,lowercase,belt,O
1021,1021,.,__end1__,__end2__,__END2__,wildcard,__END2__,__END1__,wildcard,__END1__,.,O,belt,NN,O,bulki,JJ,lowercase,bulky,lowercase,belt,43,punct,.,O


In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [6]:
class SentenceGetter(object):

  def __init__(self,data):
    self.n_sent=1
    self.data=data
    self.empty=False
    agg_fun=lambda s: [(w,p,t) for w,p,t in zip(s["word"].values.tolist(),s["pos"].values.tolist(),s["tag"].values.tolist())]                                   
    self.grouped=self.data.groupby("sentence_idx").apply(agg_fun)
    self.sentences=[s for s in self.grouped]

  def get_next(self):
    try:
      s=self.grouped["Sentence:{}".format(self.n_sent)]
      self.n_sent += 1
      return s
    
    except:
      return None

In [7]:
getter=SentenceGetter(data)

In [8]:
sentences=[[word[0] for word in sentence] for sentence in getter.sentences ]
sentences[0]

['Thousands',
 'of',
 'demonstrators',
 'have',
 'marched',
 'through',
 'London',
 'to',
 'protest',
 'the',
 'war',
 'in',
 'Iraq',
 'and',
 'demand',
 'the',
 'withdrawal',
 'of',
 'British',
 'troops',
 'from',
 'that',
 'country',
 '.']

In [9]:
 labels=[[s[2] for s in sentence] for sentence in getter.sentences]
 print(labels[0])

['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


In [10]:
#map the tags with integers
tag_values=list(set(data["tag"].values))
tag_values.append("PAD")
tag2idx={t:i for i,t in enumerate(tag_values)}

In [11]:
print(tag2idx)

{'I-per': 0, 'B-geo': 1, 'B-gpe': 2, 'B-per': 3, 'I-art': 4, 'I-org': 5, 'I-geo': 6, 'O': 7, 'B-tim': 8, 'B-org': 9, 'B-art': 10, 'PAD': 11}


In [12]:
pip install transformers

In [13]:
#Apply the Bert
import torch
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler
from transformers import BertTokenizer,BertConfig


In [14]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [15]:
MAX_LEN = 75
Batch_Size = 32

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu=torch.cuda.device_count()

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

In [ ]:
#split words into sub toekns
#We have to deal with the issue of splitting our token-level labels to related subtokens

In [18]:
def token_and_preserve_labels(sentence,text_labels):
  tokenized_sentence=[]
  labels=[]

  for word,label in zip(sentence,text_labels):
    #tokenize the word and count of # of subwords that word is toeknized into 
    tokenized_word=tokenizer.tokenize(word)
    n_subwords=len(tokenized_word)

    # Add the tokenized word to the final tokenized word list
    tokenized_sentence.extend(tokenized_word)

    # Add the same label to the new list of labels `n_subwords` times
    labels.extend([label] * n_subwords)

  return tokenized_sentence,labels 


In [20]:
#sentence and corresponsing labels 
[token_and_preserve_labels(sent,labels) for sent,labels in zip(sentences,labels)]

[(['Thousands',
   'of',
   'demons',
   '##tra',
   '##tors',
   'have',
   'marched',
   'through',
   'London',
   'to',
   'protest',
   'the',
   'war',
   'in',
   'Iraq',
   'and',
   'demand',
   'the',
   'withdrawal',
   'of',
   'British',
   'troops',
   'from',
   'that',
   'country',
   '.'],
  ['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-geo',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-geo',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-gpe',
   'O',
   'O',
   'O',
   'O',
   'O']),
 (['Families',
   'of',
   'soldiers',
   'killed',
   'in',
   'the',
   'conflict',
   'joined',
   'the',
   'protesters',
   'who',
   'carried',
   'banner',
   '##s',
   'with',
   'such',
   'slogan',
   '##s',
   'as',
   '"',
   'Bush',
   'Number',
   'One',
   'Terror',
   '##ist',
   '"',
   'and',
   '"',
   'Stop',
   'the',
   'Bomb',
   '##ings',
   '.',
   '"'],
  ['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',

In [21]:
# split text and labels
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
#labels
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

Next, we cut and pad the token and label sequences to our desired length.